In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [2]:
def get_video_list(keyword):
    url = "https://www.youtube.com/results?search_query="+keyword+"&sp=CAI%253D"
    driver = webdriver.Chrome('/Users/leesuyeon/Downloads/chromedriver')
    driver.get(url)
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    titles=[]
    urls=[]
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3.0)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_page_height == last_page_height:
            break
        last_page_height = new_page_height
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    datas = soup.select("a#video-title")
    for data in datas:
        title = data.text.replace('\n', '')
        url = "https://www.youtube.com/" + data.get('href')
        titles.append(title)
        urls.append(url)
    return titles, urls

In [6]:
def get_user_IDs_and_comments(urls):
    my_dataframe = pd.DataFrame()
    
    for url in urls:
        driver = webdriver.Chrome('/Users/leesuyeon/Downloads/chromedriver')
        driver.get(url)
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1.5)

            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(1.5)
        #유튜브 팝업 때문에 안 된다면
        # try:
        #     driver.find_element_by_css_selector("#dismiss-button > a").click()
        # except:
        #     pass

        #대댓글 버튼 누르기
        buttons = driver.find_elements_by_css_selector("#more-replies > a")

        time.sleep(1.5)

        for button in buttons:
            try:
                button.send_keys(Keys.ENTER)
                time.sleep(1.5)
                button.click()
            except:
                pass
        
        #댓글 가져오기
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        id_list = soup.select("div#header-author > h3 > #author-text > span")
        comment_list = soup.select("yt-formatted-string#content-text")

        id_final = []
        comment_final = []

        for i in range(len(comment_list)):
            temp_id = id_list[i].text
            temp_id = temp_id.replace('\n', '')
            temp_id = temp_id.replace('\t', '')
            temp_id = temp_id.replace('    ', '')
            id_final.append(temp_id) # 댓글 작성자

            temp_comment = comment_list[i].text
            temp_comment = temp_comment.replace('\n', '')
            temp_comment = temp_comment.replace('\t', '')
            temp_comment = temp_comment.replace('    ', '')
            comment_final.append(temp_comment) # 댓글 내용

        pd_data = pd.DataFrame({"ID":id_final, "Comment":comment_final})
        my_dataframe.append(pd_data)
        
    return my_dataframe

In [4]:
titles, urls = get_video_list('민식이법')
print("title len",len(titles))
my_dataframe = get_user_IDs_and_comments(urls)

/var/folders/82/5zkl1yw93n70v5n8xzdg4s140000gn/T/ipykernel_8410/625348981.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/leesuyeon/Downloads/chromedriver')


title len 476


In [ ]:
import sqlite3
con = sqlite3.connect('C:/jocoding/youtube.db')
my_dataframe.to_sql('test',con)